In [1]:
%load_ext lab_black

In [2]:
import math
import os
import random
from collections import defaultdict
from datetime import datetime
from io import TextIOWrapper
from pathlib import Path
from typing import Callable, List, Sequence, Tuple

import albumentations as A
import cv2
import imageio
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from albumentations.pytorch import ToTensorV2
from efficientnet_pytorch import EfficientNet
from resnest.torch import resnest50, resnest101, resnest200, resnest269
from sklearn.model_selection import KFold
from torch import Tensor, nn
from torch.optim import AdamW
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.dataset import Subset
from torch.utils.tensorboard import SummaryWriter
from torchvision.models import mobilenet_v2
from torchvision.models.detection import KeypointRCNN, keypointrcnn_resnet50_fpn
from torchvision.ops import MultiScaleRoIAlign
import torchvision
from tqdm import tqdm

import utils

In [3]:
DATA_DIR = Path("../data/ori")
FOLD = 1
START_EPOCH = 1
NUM_EPOCHS = 100
N_TTA_TEST = 10
N_TTA_VALID = 1
SAM = False
LR = 1e-4

In [4]:
model_m = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

In [5]:
model_k = torchvision.models.detection.keypointrcnn_resnet50_fpn(pretrained=True)

In [6]:
model_k

KeypointRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(640, 672, 704, 736, 768, 800), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256)
          (relu): ReLU(inplace=Tr

이미 잘 학습된 17개를 버리기 아까운데

나머지는 같고 5개만 추가되는 거라면 5개만 찾는 predictor를 따로 만들어서 둘을 합쳐주는게 어떨지?

근데 그러면 각각의 키포인트끼리의 관계를 무시하는게 되지 않을까?

아니면 그 keypoint predictor를 아예 새로 만들고, COCO 데이터셋에 추가 학습??

아니면 weight를 17까지만 유지하고 5층을 더 쌓는건?  
초기화는 모듈 만들 때 자동으로 하고, 0:17 까지만 기존 weight를 넣는 것.  
row 별로 requires_grad를 지정해줄 수 있으려나? --> 가능


In [14]:
model_k.roi_heads.keypoint_predictor.kps_score_lowres.weight.shape

torch.Size([512, 17, 4, 4])

In [15]:
model_k.roi_heads.keypoint_predictor.kps_score_lowres.weight.shape

torch.Size([512, 17, 4, 4])

In [16]:
m = nn.ConvTranspose2d(512, 22, 4, 2, 1)

In [17]:
m.weight.shape

torch.Size([512, 22, 4, 4])

In [18]:
m.weight[0, 0]

tensor([[-0.0354, -0.0095, -0.0401, -0.0261],
        [-0.0092, -0.0396,  0.0307,  0.0455],
        [-0.0358,  0.0482,  0.0096, -0.0511],
        [-0.0318, -0.0190,  0.0230,  0.0279]], grad_fn=<SelectBackward>)

In [19]:
m.weight[:, :17] = model_k.roi_heads.keypoint_predictor.kps_score_lowres.weight

In [20]:
m.weight[0, 0]

tensor([[ 0.0002, -0.0034, -0.0082,  0.0009],
        [-0.0038, -0.0048, -0.0038, -0.0032],
        [ 0.0012,  0.0011, -0.0037,  0.0032],
        [ 0.0042, -0.0015, -0.0011,  0.0021]], grad_fn=<SelectBackward>)

In [22]:
m.bias.shape

torch.Size([22])

In [23]:
m.bias

Parameter containing:
tensor([ 0.0415,  0.0417, -0.0169,  0.0365, -0.0030, -0.0349, -0.0036, -0.0217,
        -0.0490, -0.0279, -0.0036, -0.0516,  0.0294, -0.0361,  0.0440,  0.0015,
         0.0413, -0.0157,  0.0071, -0.0445, -0.0461, -0.0300],
       requires_grad=True)

In [24]:
model = keypointrcnn_resnet50_fpn(pretrained=True, progress=False)
for p in model.parameters():
    p.requires_grad = False

m = nn.ConvTranspose2d(512, 24, 4, 2, 1)
m.weight[:, :17] = model.roi_heads.keypoint_predictor.kps_score_lowres.weight
m.bias[:17] = model.roi_heads.keypoint_predictor.kps_score_lowres.bias
model.roi_heads.keypoint_predictor.kps_score_lowres = m

In [36]:
model.backbone.body.conv1.weight.requires_grad

False

In [8]:
df = pd.read_csv("data/ori/train_df.csv")

In [10]:
df.columns

Index(['image', 'nose_x', 'nose_y', 'left_eye_x', 'left_eye_y', 'right_eye_x',
       'right_eye_y', 'left_ear_x', 'left_ear_y', 'right_ear_x', 'right_ear_y',
       'left_shoulder_x', 'left_shoulder_y', 'right_shoulder_x',
       'right_shoulder_y', 'left_elbow_x', 'left_elbow_y', 'right_elbow_x',
       'right_elbow_y', 'left_wrist_x', 'left_wrist_y', 'right_wrist_x',
       'right_wrist_y', 'left_hip_x', 'left_hip_y', 'right_hip_x',
       'right_hip_y', 'left_knee_x', 'left_knee_y', 'right_knee_x',
       'right_knee_y', 'left_ankle_x', 'left_ankle_y', 'right_ankle_x',
       'right_ankle_y', 'neck_x', 'neck_y', 'left_palm_x', 'left_palm_y',
       'right_palm_x', 'right_palm_y', 'spine2(back)_x', 'spine2(back)_y',
       'spine1(waist)_x', 'spine1(waist)_y', 'left_instep_x', 'left_instep_y',
       'right_instep_x', 'right_instep_y'],
      dtype='object')

In [12]:
(len(df.columns) - 1) // 2

24

In [39]:
len(list(Path("data/ori/train_imgs/").glob("*.jpg")))

4195

In [41]:
imageio.imread("data/ori/train_imgs/001-1-1-01-Z17_A-0000001.jpg").shape

(1080, 1920, 3)

In [44]:
index = 0
keypoints = df.iloc[index, 1:].values.reshape(-1, 2).astype(np.int64)

In [48]:
np.concatenate([keypoints, np.ones((24, 1))], axis=1).shape

(24, 3)

In [50]:
np.concatenate([keypoints, np.ones((24, 1))], axis=1)[np.newaxis].shape

(1, 24, 3)